In [1]:
import pandas as pd
import numpy as np
import re 


In [2]:
# regular expression for validating an Email 
#Logic utilized from here for the email regex expression. It was the best working one that worked across 90k plus emails
#https://emailregex.com/
regex = r"(^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$)"

  


# We will be checking the row length after each operation for better clarity regarding the number of ops performed

## After importing the data file the Order of Operations are


1. Dropping all Empty Fields
2. Dropping all Fields with Multiple Empty Characters
3. Changing all to lowercase
4. Splitting and extracting multiple emails from the fields
5. Ensuring emails are unique in each field
6. Creating new rows for multi emails
7. Dropping all duplicates
8. Email Validation. If valid- > leave as it is. If not. Drop the row






In [4]:
raw_data = pd.read_excel("lever.xlsx")
new_data = pd.read_excel("invalids.xlsx")
df = pd.DataFrame(raw_data)
df2 = pd.DataFrame(new_data)
row_length, _ = df.shape
print("Row length -->",row_length)

Row length --> 96965


In [5]:
df_all = df.copy()

### Dropping all the Empty Fields below and also the ones with multiple empty characters

In [6]:
#Checking for empty and throwing away value
#Here NaN represents empty or you can say null
#df.dropna(subset=['Email'],inplace=True)
#ALso check for empty strings after this. Just in case
#Below is code for regex check for longer empty string like '     '
#df.replace(r'^\s*$', np.nan, regex=True)
#df.dropna(subset=["Email"],inplace=True)
#row_length, _ = df.shape
#print("Row length -->",row_length)

### Change all to lowercase

In [7]:
df['Email'] = df['Email'].str.lower()

## Few helper functions that come in handy later on for data transformations

In [8]:
#Creating a few helper functions for repeated use
#Trims and splits based on spaces
def despacer_decomma(x):
    x = x.strip()
    x = x.split(",")
    x = " ".join(x)
    x = x.split(" ")
    return x
'''
For removing false positives and weird chars in btwn valid emails, if the user uses some weird kind of characters for separating his email like ;; or -x- or ---- 
The smallest email can be of the size 6 characters. Using that heuristic we use the below logic to clear all the false positives
'''
def remover(x):
    if len(x) < 6:
        return None
    else:
        return x

def size_based_email_cleaner(l):
    l = list(map(remover,l))
    l = list(filter(None,l))
    return l

# Define a function for 
# for validating an Email 
def check(email):  
    # pass the regular expression 
    # and the string in search() method 
    if(re.search(regex,email)):  
        return True
          
    else:  
        return False 

### Email Splitting and Cleaning with the same field. Also remove case insensitive duplicates here

In [13]:
for i in df.index:
    val = str(df.at[i, "Email"])
    val1 = despacer_decomma(val)
    val2 = list(set(size_based_email_cleaner(val1)))
    df.at[i, "Email"] = val2

### Taking those multiple emails and creating multiple entries from them

Using logic below and combine with merger what can be done repeatedly is copy row where criteria is not matched, 
put the appended row in another dataframe, then remove all such rows from the original one, now you have two datasets. Instead of pure dropping what you have now is segregation of rows.
Logic below is

if validation-condition is not matched:
    duplicate row
    append that row to new dataframe
now you have df1 with all rows and df2 with only rows that didnt match your conditions
Now save separately


In [14]:
for i in df.index:
    val = df.at[i, "Email"]
    if len(val)>1:
        core = val[0]
        if type(val) is type(" "):
            continue
        while len(val) > 1:
            temp_val = val.pop()
            new_df = df.loc[i, :]
            new_df.at["Email"] = str(temp_val)
            df = df.append(new_df,ignore_index=True)
    #Uncomment below row to see each email that is appended and new row that is created
   #         print("New Row Duplicated with Following Email :-",new_df["Email"])
        df.at[i,"Email"] = core
#Converting the list datatype in the col to a string one
for i in df.index:
    df.at[i, "Email"] = "".join(df.at[i, "Email"])
row_length, _ = df.shape
print("Row length -->",row_length)

Row length --> 104300


### Dropping all the duplicate values from the entire sheets with Email as Primary Key

In [15]:
df = df.drop_duplicates(subset = ["Email"])
row_length, _ = df.shape
print("Row length -->",row_length)

Row length --> 94941


In [16]:
new_df = df.iloc[:2, :]
for i in df.index:
    val = df.at[i, "Email"]
    if check(val):
        continue
    else:
        temp_df = df.loc[i, :]
        new_df = new_df.append(temp_df,ignore_index=True)
        print("This email is invalid",val)
        df.at[i, "Email"] = np.nan
print("Cleaning Completed")
df.dropna(subset=['Email'],inplace=True)
row_length, _ = df.shape
print("Row length -->",row_length)

This email is invalid 
This email is invalid maggiedrc@comcast
This email is invalid olgawiley@kk
This email is invalid krvssiewoods@gmailcom
This email is invalid 978-609-6964|htruong@vcom.vt.edu
This email is invalid wcmust2@email
This email is invalid mesyoung@gmail
This email is invalid 516-343-5948*deborahastein@gmail.com
This email is invalid aabreu@clinicasmidoctor
This email is invalid caphr@earthlink
This email is invalid ?faizakemc@gmail.com
This email is invalid ?docdeepa_rani128@yahoo.com
This email is invalid ?bkoperski@llu.edu
This email is invalid ?eliastzele@yahoo.com
This email is invalid ?jshostetter@gmail.com
This email is invalid ?rakhshandaakram@gmail.com
This email is invalid ?mohsinhaseeb@hotmail.com
This email is invalid ?jwatson@mmc.edu
This email is invalid ?bgebhardt@svhs.org
This email is invalid ?naureenzafarmd@gmail.com
This email is invalid ?mlivny@gmail.com
This email is invalid ?drariar@gmail.com
This email is invalid ?dansavianomd@aol.com
This email is

## Ignore below code, experimental for saving and merge combinations for valid and invalid

In [18]:
new_df.to_excel("invalids.xlsx",index=False)

In [42]:
unclean_data = df_all.merge(df, how = 'outer' ,indicator=True,on=["Email"]).loc[lambda x : x['_merge']=='left_only']

In [43]:
unclean_data

,Lever ID_x,Candidate Name_x,Company_x,Email,Phone_x,Schools_x,Location_x,Links_x,Files_x,Candidate Owner Name_x,...,Schools_y,Location_y,Links_y,Files_y,Candidate Owner Name_y,Candidate Owner Email_y,Origin_y,Sources_y,Tags_y,_merge
0,0725007e-d533-494a-9ac3-32ae167b3bc8,Christopher Waguespack,"Montgomery, AL - Physicians - Family Practice ...",NaN,(612) 231-6899,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
1,f5063c21-254c-43c0-a65c-ce4504e4a937,Genova Stearns,NaN,NaN,"239-649-4242, 239-821-5521",NaN,"NAPLES, FL",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
2,67e508ca-96bd-4eea-a750-854461981a34,Robert L. Mandiberg,NaN,NaN,510-853-4128,NaN,"Emeryville, California",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
3,6a24a43c-9e16-44b0-877c-6c76e3a99242,William Boudreau,NaN,NaN,"6073459222, 6077533438",NaN,"Cortland, NY",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
4,083b4ae6-1b9e-459c-8d95-b150f09faf16,Temitayo Akindele,NaN,NaN,NaN,NaN,"Vallejo, CA",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96942,68cca3fb-3828-4917-8104-d6ec556a36aa,Syed Ahmed,"Sacramento, CA - Physicians","moiz1491@gmail.com, smahmed@som.umaryland.edu",917-821-4180,National University of Sciences and Technology,"Sacramento, CA",NaN,"Benefits Outline New Physician 2020.docx, CV.p...",Tiffany Volker,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
96958,7966e5f4-1403-4e43-9724-b8336dcc245f,Maria Tempera,"Wilkes-Barre, Pa",Tempera.Maria@gmail.com,(908) 591-6092,"Overlook Medical Center, Rowan University Scho...","Wilkes-Barre, Pa",NaN,"CV -1320.pdf, CV .pdf, Physician Employment Ag...",Mitch Diamond,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
96959,e5c4642b-2c7f-4606-9eb8-40c53a59f1cc,Seif Atyia,"Sacramento, CA","seifatyia@gmail.com, seifatyia@yahoo.com",(423) 426-2231,"The University of Tennessee, The University of...","Sacramento, CA",NaN,"ALL - Benefits Outline New Physician 2020.pdf,...",Noelia Alvarez,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
96960,75859464-b5f7-498b-8695-515608af314a,Emily Krajec,"Mount Carmel, IL",Drekrajec@gmail.com,(618) 240-5770,NaN,"Mount Carmel, IL 62863",NaN,NaN,Erin Wiederkehr,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only


In [18]:
df = checkpoint.copy()

,Lever ID,Candidate Name,Company,Email,Phone,Schools,Location,Links,Files,Candidate Owner Name,Candidate Owner Email,Origin,Sources,Tags
0,a0c9f94e-e81a-482f-8c54-ad3bb4bc99c4,Urvashi Mehta,"Fort Mill, SC - Physicians - Hospitalist",md_uhmehta@yahoo.com,803.431.3966,NaN,Fort Mill,NaN,NaN,NaN,NaN,applied,"Practice Alert, iCIMS","Hospitalist, Physicians"
1,8c639933-4959-4dbf-9230-dc18475e07cf,Bruce Barach,"schenectady, NY - Physicians - Plastic Surgeon",brucebarach@aol.com,(518) 346-3125,"Dartmouth College, S.U.N.Y. Health Science Cen...",1201 nott st #303,NaN,NaN,NaN,NaN,applied,"Practice Alert, iCIMS","Physicians, Plastic Surgeon"
2,5a49e945-f5a2-49b5-9235-d581f5667f42,Neal Dollin,"Lebanon, OH - Physicians - PGY",neal.dollin@gmail.com,(513) 910-5224,NaN,NaN,NaN,NaN,NaN,NaN,applied,"APDS.org, iCIMS","PGY- Surgeon who did not complete residency, P..."
3,1f87c619-d6f1-4ed9-9546-3f519d489cd8,Venkat Yalamanchili,"Houston, TX - Physicians - Vascular Surgeon",rvy@hotmail.com,(281) 236-3587,NaN,4406 Orange Leaf Court,NaN,NaN,NaN,NaN,applied,"Practicelink, iCIMS","Physicians, Vascular Surgeon"
4,9a753fe4-4b7e-4698-8be1-5643e8082d26,"Obaydah AbdurRaqeeb,","cocoa, FL - Physicians - Family Practice Medicine",obaydah@hotmail.com,(865) 964-7105,NaN,NaN,NaN,NaN,NaN,NaN,applied,"Practicelink, iCIMS","Family Practice Medicine, Physicians"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102916,28c80667-daef-4dfe-af4f-9cd12de4c320,Howard G. Stern,"Richmond, VA",howardsternmd@gmail.com,"(410) 933-5678, (804) 272-3300, 804-513-4585","City University of New York, MEDICAL SCHOOL Me...","9601 Pulaski Park Drive,Suite 416, Baltimore, ...",NaN,"ALL - Benefits Outline 2019.pdf, ALL - LewinSt...",Erin Wiederkehr,ewiederkehr@vohraphysicians.com,applied,"Email, MDJobSite, iCIMS","API, Board Certified, Orthopoedic Surgeon, Phy..."
102922,89c93047-c86b-4f36-a160-196d3bcf15f1,Katherine Lee,"Sacramento, CA - Physicians",katbolee66@gmail.com,216-374-3675,"Kent State University, Northeastern Ohio Unive...","Sacramento, CA",https://vohraphysicians.box.com/s/k6mcqqyyeufs...,"KLee - Background 2 - 9.docx, KLee - NPDB.pdf,...",Erin Wiederkehr,ewiederkehr@vohraphysicians.com,applied,Organic,"API, Internal Medicine, Physicians"
102923,de7d6702-60a4-4801-8f57-03abd9fe83cc,Winni Loesch,"Grass Valley, CA",drloesch@amethystmed.com,'+15302778274,NaN,"Grass Valley, CA",NaN,"ALL-BenefitsOutlineNewPhysician2020.pdf, ALL-L...",Erin Wiederkehr,ewiederkehr@vohraphysicians.com,applied,Facebook,"API, Physicians"
102924,68cca3fb-3828-4917-8104-d6ec556a36aa,Syed Ahmed,"Sacramento, CA - Physicians",moiz1491@gmail.com,917-821-4180,National University of Sciences and Technology,"Sacramento, CA",NaN,"Benefits Outline New Physician 2020.docx, CV.p...",Tiffany Volker,tvolker@vohraphysicians.com,applied,Organic,"API, PGY- Surgeon who did not complete residen..."


### Saving the file to the memory

In [44]:
df.to_excel("email.xlsx",index=False)

#### Disclaimer :- For running this out of the box :- delete all column headers before calling the file, and paste back after wrangling. There is some weird bug that corrupts the dataframe in pandas